In [2]:
import pandas as pd 
import pymorphy3 as pm
import re

cleaned_data = pd.read_csv("kp_final.csv")

morph = pm.MorphAnalyzer()

def string_preparation(string:str) -> str:
    mask = r'[^\w\s]'
    
    sub = re.sub(mask, '', string.lower())
    return sub


def normalize_string(string: str) -> str:
    test_str = string_preparation(string=string)

    string = ''
    words = test_str.split()

    for i in words:
        s = morph.parse(i)
        
        string += s[0].normal_form + ' '
        
    return string.strip()
        
#print(normalize_string(cleaned_data.overview[0]))


In [3]:
data = {'movie':[i for i in cleaned_data.movie], 'overview':[normalize_string(i) for i in cleaned_data.overview]}
new_data = pd.DataFrame(data=data)

new_data.to_csv('normalized.csv')

In [1]:
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity
import pandas as pd
import numpy as np
import requests
import json
from config import sys_prompt
import pymorphy3 as pm


normalized = pd.read_csv('normalized.csv')

morph = pm.MorphAnalyzer()

vectorizer = TfidfVectorizer(analyzer='word', ngram_range=(1, 2))

tfidf_matrix = vectorizer.fit_transform(normalized['overview'])

user_queue = input()



s = ''

for i in user_queue.split():
    s+=morph.parse(i)[0].normal_form+' '
    
print(s)    

query_vector = vectorizer.transform([s])

# 3. Вычисление косинусного сходства
# Сравниваем вектор запроса со всеми векторами из нашей матрицы документов
cosine_similarities = cosine_similarity(query_vector, tfidf_matrix)
similarity_scores = cosine_similarities[0]

# Находим все индексы с ненулевым сходством
non_zero_indices = np.where(similarity_scores > 0.1)[0]
non_zero_scores = similarity_scores[non_zero_indices]

# Сортируем по убыванию сходства и берем не более 5 лучших
sorted_indices = non_zero_indices[np.argsort(non_zero_scores)[::-1]]
top_indices = sorted_indices[:5]  # Берем максимум 5
top_scores = similarity_scores[top_indices]

print(f"\nЗапрос пользователя: '{user_queue}'")

if len(top_indices) > 0:
    print(f"Наиболее подходящие товары (найдено {len(top_indices)}):")
    
    for i, (idx, score) in enumerate(zip(top_indices, top_scores), 1):
        best_match_title = normalized.loc[idx, 'movie']
        print(f"{i}. '{best_match_title}' (сходство: {score:.4f})")
else:
    print("Не найдено подходящих товаров с ненулевым сходством.")


привет 

Запрос пользователя: 'Привет'
Не найдено подходящих товаров с ненулевым сходством.


In [4]:
import gensim.downloader as api
import nltk

fasttext_model300 = api.load('fasttext-wiki-news-subwords-300')

[===-----------------------------------------------] 6.1% 58.8/958.4MB downloaded

KeyboardInterrupt: 